In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.4 MB/s eta 0:00:00


In [ ]:
question = "What are the capabilities of Hugging Face"
context = "Hugging Face offers a set of natural language processing (NLP) features, including the popular open-source library transformers."

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
question_answerer(question=question, context=context)

{'score': 0.11298573762178421,
 'start': 29,
 'end': 71,
 'answer': 'natural language processing (NLP) features'}

In [ ]:
## load your sentiment analysis dataset for finetuning
from datasets import load_dataset

sentiment_dataset = load_dataset("rotten_tomatoes")


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(“bert-base-uncased”)


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = sentiment_dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"].shuffle(seed=10).select(range(1000))
eval_dataset = tokenized_datasets["validation"].shuffle(seed=10).select(range(1000))


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


In [ ]:
from transformers import TrainingArguments
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer")

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
